In [30]:
import rasterio as rio
import os

import boto3
import threading
import sys

In [9]:
root = "/Users/nathansuberi/Desktop/RW_Data/CHIRPS"
os.chdir(root)

In [48]:
folders = os.listdir()[1:]
folders

['CHIRPS_spi_2006-01-01_2006-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E',
 'CHIRPS_spi_2007-01-01_2007-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E',
 'CHIRPS_spi_2008-01-01_2008-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E',
 'CHIRPS_spi_2009-01-01_2009-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E',
 'CHIRPS_spi_2010-01-01_2010-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E',
 'CHIRPS_spi_2011-01-01_2011-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E',
 'CHIRPS_spi_2012-01-01_2012-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E',
 'CHIRPS_spi_2013-01-01_2013-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E',
 'CHIRPS_spi_2014-01-01_2014-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E',
 'CHIRPS_spi_2015-01-01_2015-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E',
 'merged_SPI_data.tif']

In [10]:
files = [root + '/' + file + '/' + file + ".precipitation.tif" for file in folders]


In [49]:
files

['/Users/nathansuberi/Desktop/RW_Data/CHIRPS/CHIRPS_spi_2006-01-01_2006-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E/CHIRPS_spi_2006-01-01_2006-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E.precipitation.tif',
 '/Users/nathansuberi/Desktop/RW_Data/CHIRPS/CHIRPS_spi_2007-01-01_2007-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E/CHIRPS_spi_2007-01-01_2007-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E.precipitation.tif',
 '/Users/nathansuberi/Desktop/RW_Data/CHIRPS/CHIRPS_spi_2008-01-01_2008-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E/CHIRPS_spi_2008-01-01_2008-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E.precipitation.tif',
 '/Users/nathansuberi/Desktop/RW_Data/CHIRPS/CHIRPS_spi_2009-01-01_2009-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E/CHIRPS_spi_2009-01-01_2009-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E.precipitation.tif',
 '/Users/nathansuberi/Desktop/RW_Data/CHIRPS/CHIRPS_

In [23]:
with rio.open(files[0], 'r') as src:
    profile = src.profile
    print(profile)

{'driver': 'GTiff', 'dtype': 'float64', 'nodata': None, 'width': 8350, 'height': 2320, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': Affine(0.043119133637737034, 0.0, -180.02238293755212,
       0.0, -0.043119133637737034, 50.01819501977496), 'tiled': False, 'interleave': 'band'}


In [19]:
file_edits = [file[:-4] + "_edit.tif" for file in files]
merge_file = root + '/merged_SPI_data.tif'
band_names = list(range(2006, 2016))

In [27]:
profile_edit = profile.copy()
profile_edit.update(compress="lzw")

# Create compressed edit files
for ix, file in enumerate(files):
    print(file)
    with rio.open(file, "r") as src:
        with rio.open(file_edits[ix], "w", **profile_edit) as dst:
            src_data = src.read(indexes=1)
            dst.write(src_data, indexes=1)

/Users/nathansuberi/Desktop/RW_Data/CHIRPS/CHIRPS_spi_2006-01-01_2006-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E/CHIRPS_spi_2006-01-01_2006-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E.precipitation.tif
/Users/nathansuberi/Desktop/RW_Data/CHIRPS/CHIRPS_spi_2007-01-01_2007-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E/CHIRPS_spi_2007-01-01_2007-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E.precipitation.tif
/Users/nathansuberi/Desktop/RW_Data/CHIRPS/CHIRPS_spi_2008-01-01_2008-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E/CHIRPS_spi_2008-01-01_2008-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E.precipitation.tif
/Users/nathansuberi/Desktop/RW_Data/CHIRPS/CHIRPS_spi_2009-01-01_2009-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E/CHIRPS_spi_2009-01-01_2009-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E.precipitation.tif
/Users/nathansuberi/Desktop/RW_Data/CHIRPS/CHIRPS_spi_2010-01-01_201

In [28]:
profile_merge = profile.copy()
profile.update(count=len(files), compress="lzw", bands = band_names)

with rio.open(merge_file, "w", **profile) as dst:
    for ix, file in enumerate(file_edits):
        print(file)
        with rio.open(file, "r") as src:
            src_data = src.read(indexes=1)
            dst.write(src_data, indexes=ix+1)

/Users/nathansuberi/Desktop/RW_Data/CHIRPS/CHIRPS_spi_2006-01-01_2006-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E/CHIRPS_spi_2006-01-01_2006-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E.precipitation_edit.tif
/Users/nathansuberi/Desktop/RW_Data/CHIRPS/CHIRPS_spi_2007-01-01_2007-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E/CHIRPS_spi_2007-01-01_2007-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E.precipitation_edit.tif
/Users/nathansuberi/Desktop/RW_Data/CHIRPS/CHIRPS_spi_2008-01-01_2008-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E/CHIRPS_spi_2008-01-01_2008-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E.precipitation_edit.tif
/Users/nathansuberi/Desktop/RW_Data/CHIRPS/CHIRPS_spi_2009-01-01_2009-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E/CHIRPS_spi_2009-01-01_2009-12-31_vs_1981-2006_-50p0000to50p0000N_180p0000to-180p0000E.precipitation_edit.tif
/Users/nathansuberi/Desktop/RW_Data/CHIRPS/CHIRP

In [29]:
with rio.open(merge_file, "r") as src:
    print(src.profile)

{'driver': 'GTiff', 'dtype': 'float64', 'nodata': None, 'width': 8350, 'height': 2320, 'count': 10, 'crs': CRS({'init': 'epsg:4326'}), 'transform': Affine(0.043119133637737034, 0.0, -180.02238293755212,
       0.0, -0.043119133637737034, 50.01819501977496), 'bands': '[2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]', 'compress': 'lzw', 'interleave': 'band', 'tiled': False}


In [31]:
s3_upload = boto3.client("s3")
s3_download = boto3.resource("s3")

s3_bucket = "wri-public-data"
s3_folder = "resourcewatch/raster/cli_023_CHIRPS_SPI/"

s3_key_merge = s3_folder + "cli_023_CHIRPS_SPI_2006_to_2015_annual_merge.tif"

class ProgressPercentage(object):
        def __init__(self, filename):
            self._filename = filename
            self._size = float(os.path.getsize(filename))
            self._seen_so_far = 0
            self._lock = threading.Lock()

        def __call__(self, bytes_amount):
            # To simplify we'll assume this is hooked up
            # to a single filename.
            with self._lock:
                self._seen_so_far += bytes_amount
                percentage = (self._seen_so_far / self._size) * 100
                sys.stdout.write("\r%s  %s / %s  (%.2f%%)"%(
                        self._filename, self._seen_so_far, self._size,
                        percentage))
                sys.stdout.flush()

In [32]:
# Edit
s3_upload.upload_file(merge_file, s3_bucket, s3_key_merge,
                         Callback=ProgressPercentage(merge_file))

/Users/nathansuberi/Desktop/RW_Data/CHIRPS/merged_SPI_data.tif  507365056 / 507365056.0  (100.00%)

In [33]:
os.environ["s3_key"] = "s3://" + s3_bucket + "/" + s3_key_merge
os.environ["gs_key"] = "gs://resource-watch-public/" + s3_key_merge
os.environ["asset_id"] = "users/resourcewatch/cli_023_CHIRPS_2006_to_2015_annual_SPI"

!gsutil cp $s3_key $gs_key

Non-MD5 etag ("1dab5c7e0e0344ffe98bdfde63656e20-61") present for key <Key: wri-public-data,resourcewatch/raster/cli_023_CHIRPS_SPI/cli_023_CHIRPS_SPI_2006_to_2015_annual_merge.tif>, data integrity checks are not possible.
Copying s3://wri-public-data/resourcewatch/raster/cli_023_CHIRPS_SPI/cli_023_CHIRPS_SPI_2006_to_2015_annual_merge.tif [Content-Type=binary/octet-stream]...
| [1 files][483.9 MiB/483.9 MiB]   10.9 MiB/s                                   
Operation completed over 1 objects/483.9 MiB.                                    


In [34]:
!earthengine upload image --asset_id=$asset_id $gs_key

Started upload task with ID: BNRLA2TNA2PIBO7GLRBQFYWD


In [37]:
!earthengine task info BNRLA2TNA2PIBO7GLRBQFYWD

BNRLA2TNA2PIBO7GLRBQFYWD:
  State: COMPLETED
  Type: Upload
  Description: Asset ingestion: users/resourcewatch/cli_023_CHIRPS_2006_to_2015_annual_SPI
  Created: 2017-10-05 09:51:02.627000
  Started: 2017-10-05 09:51:05.831000
  Updated: 2017-10-05 10:02:54.841000


In [36]:
os.environ["bandnames"] = str(band_names)
!earthengine asset set -p "(string)bandname=$bandnames" users/resourcewatch/cli_023_CHIRPS_2006_to_2015_annual_SPI